In [ ]:
## ARIMA model predictions for synthetic pollution time series data
# by Akanksha Upadhyay
# date: 19th March 2025
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.stattools import adfuller


# I am using an artificial dataset to demonstrate the ARIMA model
# generate time series for the data

date = pd.date_range(start = '1/1/2020', end = '6/1/2020', freq = 'D')      ## mm/dd/yyyy
print(date.shape)
# Generate synthetic data for pollutants (in µg/m³ for PM10, PM2.5, SO2, NO2, O3, and mg/m³ for CO)

np.random.seed(17)

data = ({'PM10': np.random.uniform(20, 150, len(date)),
         'PM2.5': np.random.uniform(15, 100, len(date)),
         'SO2': np.random.uniform(5, 50, len(date)),
         'NO2': np.random.uniform(10, 80, len(date)),
         'O3': np.random.uniform(0.5, 4, len(date)),
         'CO': np.random.uniform(10, 120, len(date)), ## CO is typically measured in mg/m³

         #Industrial emissions (in tons/day)
         "NOx_Emissions": np.random.uniform(50, 300, len(date)),
         "SO2_Emissions": np.random.uniform(30, 200, len(date)),
         "TSP_Emissions": np.random.uniform(10, 100, len(date)),
         "Total_Waste_Gas_Emissions": np.random.uniform(1000, 5000, len(date)),
         }) 


# convert to dataframe

data = pd.DataFrame(data, index = date) 

print(data.head())
# calculate AQI for each pollutant

breakpoints = {
    "PM10": [((0, 54), (0, 50)), ((55, 154), (51, 100)), ((155, 254), (101, 150)),
    ((255, 354), (151, 200)), ((355, 424), (201, 300)), ((425, 550), (301, 400))],

    "PM2.5": [((0, 9), (0, 50)), ((9.1, 35.4), (51, 100)), ((35.5, 55.4), (101, 150)),
    ((55.5, 125.4), (151, 200)), ((125.5, 225.4), (201, 300)), ((225.5, 300), (301, 400))],

    "SO2": [((0, 35), (0, 50)), ((36, 75), (51, 100)), ((76, 185), (101, 150)),
    ((186, 304), (151, 200)), ((305, 604), (201, 300)), ((605, 700), (301, 400))],

    "NO2": [((0, 53), (0, 50)), ((54, 100), (51, 100)), ((101, 360), (101, 150)),
    ((361, 649), (151, 200)), ((650, 1249), (201, 300)), ((1250, 3000), (301, 400))],

    "CO": [((0, 4.4), (0, 50)), ((4.5, 9.4), (51, 100)), ((9.5, 12.4), (101, 150)),
    ((12.5, 15.4), (151, 200)), ((15.5, 30.4), (201, 300)), ((30.5, 70), (301, 400))], 

    "O3": [((0.000, 0.054), (0, 50)), ((0.055, 0.070), (51, 100)), ((0.071, 0.085), (101, 150)),
    ((0.086, 0.105), (151, 200)), ((0.106, 0.200), (201, 300)), ((0.201, 0.300), (301, 400))],
}


# Function to calculate IAQI for a given pollutant
# using equstion: IAQI = (IAQI_high - IAQI_low)/(BP_high - BP_low) * (C - BP_low) + IAQI_low

Pollutant = ['PM10', 'PM2.5', 'SO2', 'NO2', 'CO', 'O3']

def calculate_iaqi(pollutant, concentration):
    for bp in breakpoints[pollutant]:
        (C_low, C_high), (IAQI_low, IAQI_high) = bp  # Unpack the nested tuples
        if C_low <= concentration <= C_high:
            IAQI = (IAQI_high - IAQI_low) / (C_high - C_low) * (concentration - C_low) + IAQI_low
            return IAQI
    return None




# Calculate IAQI and AQI for the dataset

def calculate_aqi(df):
    iaqi_cols = []
    for pollutant in ["PM10", "PM2.5", "SO2", "NO2", "CO", "O3"]:
        iaqi_col = f"IAQI_{pollutant}"
        df[iaqi_col] = df[pollutant].apply(lambda x: calculate_iaqi(pollutant, x))
        iaqi_cols.append(iaqi_col)

        # Calculate the AQI as the maximum of individual IAQIs
        
    df["AQI"] = df[iaqi_cols].max(axis=1)
    return df
    
# Apply AQI calculation to the data
data = calculate_aqi(data)
print(data.head())

# divide the AQI into categories


AQI_categories = [(0,50, 'Good'), (51, 100, 'Moderate'), (101, 150, 'Unhealthy for Sensitive Groups'), (151, 200, 'Unhealthy'), 
                  (201, 300, 'Very Unhealthy'), (301, 400, 'Hazardous')]

def categorize_aqi(aqi):
    for low, high, category in AQI_categories:
        if low <= aqi <= high:
            return category
    return None

data["AQI_Category"] = data["AQI"].apply(categorize_aqi)

print(data.head())
import matplotlib.pyplot as plt

# Create subplots with 7 rows (one for each trend) and 1 column
fig, axes = plt.subplots(7, 1, figsize=(12, 14), sharex=True)

# Plot AQI trend
axes[0].plot(data.index, data["AQI"], label="AQI", color='black')
axes[0].set_title("AQI Trend")
axes[0].set_ylabel("Index")
axes[0].legend()

# Plot PM10 trend
axes[1].plot(data.index, data["PM10"], label="PM10", color='red')
axes[1].set_title("PM10 Trend")
axes[1].set_ylabel("µg/m³")
axes[1].legend()

# Plot PM2.5 trend
axes[2].plot(data.index, data["PM2.5"], label="PM2.5", color='blue')
axes[2].set_title("PM2.5 Trend")
axes[2].set_ylabel("µg/m³")
axes[2].legend()

# Plot SO2 trend
axes[3].plot(data.index, data["SO2"], label="SO2", color='green')
axes[3].set_title("SO2 Trend")
axes[3].set_ylabel("µg/m³")
axes[3].legend()

# Plot NO2 trend
axes[4].plot(data.index, data["NO2"], label="NO2", color='purple')
axes[4].set_title("NO2 Trend")
axes[4].set_ylabel("µg/m³")
axes[4].legend()

# Plot CO trend
axes[5].plot(data.index, data["CO"], label="CO", color='orange')
axes[5].set_title("CO Trend")
axes[5].set_ylabel("mg/m³")  # CO is often measured in mg/m³
axes[5].legend()

# Plot O3 trend
axes[6].plot(data.index, data["O3"], label="O3", color='brown')
axes[6].set_title("O3 Trend")
axes[6].set_ylabel("µg/m³")
axes[6].set_xlabel("Date")
axes[6].legend()

# Add gridlines to all subplots
for ax in axes:
    ax.grid()

# Adjust layout to prevent overlapping
plt.tight_layout()

# Show the plot
plt.show()

## prediction of AQI levels using ARIMA model

# Select the variable for prediction (e.g., AQI)

series = data['AQI']
print(series.head())
# check if the trend is stationary

def check_stationarity(series):
    result = adfuller(series)
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    if result[1] <= 0.05:
        print("The series is stationary (p-value <= 0.05).")
    else:
        print("The series is NOT stationary (p-value > 0.05).")

check_stationarity(series)


series = pd.to_numeric(series, errors='coerce')
print(series.dtypes)
# Apply differencing if the series iis not stationary
series_diff = series.diff().dropna()
check_stationarity(series_diff)


# determine ARIMA paramters

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plot ACF and PACF
plt.figure(figsize=(12, 6))

# ACF
plt.subplot(121)
plot_acf(series_diff, ax=plt.gca(), lags=30)
plt.title("ACF Plot")

# PACF
plt.subplot(122)
plot_pacf(series_diff, ax=plt.gca(), lags=30)
plt.title("PACF Plot")

plt.tight_layout()
plt.show()
# Set ARIMA parameters (replace p, d, q with appropriate values based on ACF/PACF)
p, d, q = 1, 1, 2  # Example values
model = ARIMA(series, order=(p, d, q))

# Fit the model
model_fit = model.fit()
print(model_fit.summary())
# Forecast the next 10 days
forecast = model_fit.forecast(steps=10)
print("Forecasted Values:")
print(forecast)

# Plot the original series and forecast
plt.figure(figsize=(12, 6))
plt.plot(series, label="Original Data")
plt.plot(forecast, label="Forecast", color='red')
plt.xlabel("Date")
plt.ylabel("AQI")
plt.title("ARIMA Forecast")
plt.legend()
plt.show()
# Calculate errors (if you have a test set to compare)
y_true = series[-10:]  # Replace with actual test data
y_pred = forecast[:len(y_true)]

mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
